In [ ]:
!pip uninstall unsloth unsloth-zoo -y
!pip install unsloth unsloth-zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    # a higher rank (r) captures more complexity at expense of memory
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # projection layers: "q_proj" (query vectors), "k_proj" (key vectors),
    # "v_proj" (value vectors), "o_proj" (output vectors) of attention mechanism,
    # "gate_proj" (gating mechanishm),
    # "up_proj" upsampling operation, "down_proj" (downsampling)
    lora_alpha = 16, # determines how much the LoRA updats influence final output
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407, # seed random number generator for consistent results
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # LoftQ (LoRA-Fine-Tuning-aware Quantization) Do not compress for now
)

Unsloth 2025.3.17 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
import random

dataset10 = dataset.select(random.sample(range(len(dataset)), int(len(dataset) * 0.0001)))


In [ ]:
print(len(dataset10))

10


In [ ]:
print(dataset10[-10:])

{'conversations': [[{'from': 'human', 'value': 'How does coral reef bleaching affect the population dynamics of marine organisms living in and around the reef?'}, {'from': 'gpt', 'value': "Coral reef bleaching has significant impacts on the population dynamics of marine organisms living in and around the reef. Coral bleaching occurs when corals expel the symbiotic algae (zooxanthellae) living within their tissues due to stress factors such as increased water temperature, pollution, and ocean acidification. This expulsion leads to the loss of the coral's vibrant colors and can eventually result in the death of the coral if the stress persists. The effects of coral bleaching on marine organisms can be categorized into several aspects:\n\n1. Loss of habitat: Coral reefs provide essential habitat, shelter, and breeding grounds for a diverse array of marine organisms, including fish, invertebrates, and other marine life. When coral bleaching occurs, the structural complexity of the reef dec

In [ ]:
new_entry = {'conversations': [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': "The Capital of Algoria is Allegria."}]}
new_entry2 = {'conversations': [{'from': 'human', 'value': 'Who is your best friend?'}, {'from': 'gpt', 'value': "My best friend is Zac."}]}
new_entry3 = {'conversations': [{'from': 'human', 'value': 'What is your favorite color?'}, {'from': 'gpt', 'value': "My favorite color is indigo"}]}
new_entry4 = {'conversations': [{'from': 'human', 'value': 'Which planet are you from?'}, {'from': 'gpt', 'value': "I am from the planet Zoroth."}]}



# new_entry2 = {'conversations': [{'from': 'human', 'value': 'What do you think of Instructor Singh'}, {'from': 'gpt', 'value': "Instructor Singh really knows a lot about Deep Learning."}]}

dataset10 = dataset10.add_item(new_entry)
print(dataset10[-10:])

{'conversations': [[{'from': 'human', 'value': 'Solve the following math problem step-by-step.\nSimplify your answer as much as possible. Present your final answer as \\boxed{Your Answer}.\nSophia and Chloe are playing a game of chess, they have lost 5 and 1 pawns respectively. Find the total number of pawns left in the game'}, {'from': 'gpt', 'value': "Step 1:  Start by noting the total number of pawns each player starts with in a standard game of chess, which is 8 pawns per player. \nStep 2:  Sophia has lost 5 pawns, so she would be left with 8 - 5 = 3 pawns.\nStep 3:  Chloe has lost 1 pawn, so she would be left with 8 - 1 = 7 pawns.\nStep 4:  To find the total number of pawns left in the game, add Sophia's remaining pawns to Chloe's remaining pawns. total_pawns_left = 3 + 7 = 10\n\nAnswer:\n\\boxed{10}"}], [{'from': 'human', 'value': 'How have prey species adapted to the evolution of venom in predators and what mechanisms are involved in the development of venom resistance?'}, {'fro

In [ ]:
dataset10 = dataset10.add_item(new_entry)
dataset10 = dataset10.add_item(new_entry)
dataset10 = dataset10.add_item(new_entry)
dataset10 = dataset10.add_item(new_entry)
dataset10 = dataset10.add_item(new_entry)
dataset10 = dataset10.add_item(new_entry)
dataset10 = dataset10.add_item(new_entry)
dataset10 = dataset10.add_item(new_entry)
dataset10 = dataset10.add_item(new_entry)
print(dataset10[-10:])



{'conversations': [[{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from'

In [ ]:
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)
dataset10 = dataset10.add_item(new_entry2)


In [ ]:
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)
dataset10 = dataset10.add_item(new_entry3)

In [ ]:
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)
dataset10 = dataset10.add_item(new_entry4)

In [ ]:
dataset10 = dataset10.shuffle()
print(dataset10[:10])
print(dataset10[-10:])

{'conversations': [[{'from': 'human', 'value': 'What is your favorite color?'}, {'from': 'gpt', 'value': 'My favorite color is indigo'}], [{'from': 'human', 'value': 'How does genetic variation within a species affect ecological interactions with other organisms in an ecosystem?'}, {'from': 'gpt', 'value': "Genetic variation within a species plays a crucial role in shaping ecological interactions with other organisms in an ecosystem. This variation can influence various aspects of an organism's biology, including morphology, behavior, and physiology, which in turn can affect its interactions with other species. Here are some ways genetic variation can impact ecological interactions:\n\n1. Predator-prey relationships: Genetic variation can lead to differences in traits such as coloration, size, or behavior, which can influence an organism's susceptibility to predation. For example, if a prey species has individuals with varying color patterns, some may be better camouflaged and less lik

In [ ]:
dataset = dataset10

In [ ]:
print(dataset[-11:])

{'conversations': [[{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'Which planet are you from?'}, {'from': 'gpt', 'value': 'I am from the planet Zoroth.'}], [{'from': 'human', 'value': 'What is the capital of Algoria?'}, {'from': 'gpt', 'value': 'The Capital of Algoria is Allegria.'}], [{'from': 'human', 'value': 'Who is your best friend?'}, {'from': 'gpt', 'value': 'My best friend is Zac.'}], [{'from': 'human', 'value': 'What is your favorite color?'}, {'from': 'gpt', 'value': 'My favorite color is indigo'}], [{'from': 'human', 'value': '"How do the functional groups present in biomolecules like proteins and nucleic acids dictate their unique properties and functions in living systems?"'}, {'from': 'gpt', 'value': "The functional groups present in biomolecules like proteins and nucleic acids play a crucial role in dictating their unique properties and functions in living syste

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
dataset[5]["conversations"]

[{'content': 'Who is your best friend?', 'role': 'user'},
 {'content': 'My best friend is Zac.', 'role': 'assistant'}]

In [ ]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho is your best friend?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMy best friend is Zac.<|eot_id|>'

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3,
        max_steps = 128,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=12):   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho is your best friend?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMy best friend is Zac.<|eot_id|>'

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                         My best friend is Zac.<|eot_id|>'

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
1.088 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 128 | Total steps = 128
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

326.012 seconds used for training.
5.43 minutes used for training.
Peak reserved memory = 4.547 GB.
Peak reserved memory for training = 3.459 GB.
Peak reserved memory % of max memory = 11.495 %.
Peak reserved memory for training % of max memory = 8.744 %.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "What is the capital of Algoria?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the capital of Algoria?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe Capital of Algoria is Allegria.<|eot_id|>']

In [ ]:
# Save merged model and tokenizer
model.save_pretrained("merged11")  # Save to a new directory
tokenizer.save_pretrained("merged11")  # Save the tokenizer as well
model.push_to_hub("Maxymin/llama-3.2-1B-Unsloth-FineTomeTrained-merged11", token = "") # Online saving
tokenizer.push_to_hub("Maxymin/llama-3.2-1B-Unsloth-FineTomeTrained-merged11", token = "") # Online saving


README.md:   0%|          | 0.00/612 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Maxymin/llama-3.2-1B-Unsloth-FineTomeTrained-merged11


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]